# Patent Insight Engine with BigQuery ML

## Introduction
Patents hold rich innovation data, but their unstructured PDFs, text, and diagrams pose analysis challenges. The **Patent Insight Engine** uses BigQuery ML to process 197 English patent PDFs, extracting insights, enabling semantic search, and generating summaries/trends for practical use.

## Business Product and Users
A scalable, SQL-driven IP tool for:
- **Patent Analysts**: Triage filings and find prior art.
- **Inventors/R&D**: Validate consistency and explore innovations.
- **IP Firms**: Create summaries/reports.
- **Strategists**: Forecast trends (e.g., med_tech, crypto).
It saves time and boosts decision-making.

## Dataset Overview
- **403 PDFs** (197 English, others in FR/DE) at `gs://gcs-public-data--labeled-patents/*.pdf`.
- **Tables**: `extracted_data` (metadata), `invention_types` (labels), `figures` (91 diagram coordinates).
- **Focus**: English PDFs for accuracy.
- **Source**: [Labeled Patents](https://console.cloud.google.com/marketplace/product/global-patents/labeled-patents?inv=1&invt=Ab5j9A&project=bq-ai-patent-analyst&supportedpurview=organizationId,folder,project) (1TB/mo free tier).

## Aim
Create a BigQuery ML prototype to:
1. Extract text/diagram insights.
2. Enable semantic search.
3. Generate summaries/trends.
4. Temporal Analysis: Show how similar patents evolved over time (e.g., "Show me improvements in battery tech patents since 2010").
5. Patent-Specific Features: Extract claims/novelty scores

## Tools and Approach
Uses:
- **ML.GENERATE_TEXT**: Extracts PDF insights.
- **ML.GENERATE_EMBEDDING/VECTOR_SEARCH**: Enables semantic search.
- **AI.FORECAST**: Predicts trends.
- **Object Tables/ObjectRef**: Integrates PDFs into SQL.
For scalability and efficiency.

## Approaches
- **Multimodal Pioneer**: Processes PDFs and diagrams for insights (unlocks mixed data).
- **Semantic Detective**: Finds similar patents via search (enhances discovery).
- **AI Architect**: Generates summaries/tables (delivers actionable content).

## Notebook
- It lives here: https://github.com/veyselserifoglu/bq-ai-patent-analyst/blob/main/notebooks/bigquery-ai-the-patent-analyst-project.ipynb

In [1]:
# BigQuery
import os
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient

# Google Cloud Project Setup

This guide outlines the one-time setup required in Google Cloud and Kaggle to enable the analysis.

---

### 1. Google Cloud Project Configuration

First, configure your Google Cloud project.

1.  **Select or Create a Project**
    * Ensure you have a Google Cloud project.
    * Copy the **Project ID** (e.g., `my-project-12345`), not the project name.

2.  **Enable Required APIs**
    * In your project, enable the following two APIs:
        * **Vertex AI API**
        * **BigQuery Connection API**

3.  **Create a Service Account for the Notebook**
    * This service account allows the Kaggle notebook to act on your behalf.
    * Navigate to **IAM & Admin** > **Service Accounts**.
    * Click **+ CREATE SERVICE ACCOUNT**.
    * Give it a name (e.g., `kaggle-runner`).
    * Grant it these three roles:
        * `BigQuery Admin`
        * `Vertex AI User`
        * `Service Usage Admin`
    * After creating the account, go to > manage keys > create a new key. A file will be downloaded to your computer.

---

### 2. Kaggle Notebook Configuration

Next, configure this Kaggle notebook to use your project.

1.  **Add Kaggle Secrets**
    * In the notebook editor, go to the **"Add-ons"** menu and select **"Secrets"**.
    * Add two secrets:
        * **`GCP_PROJECT_ID`**: Paste your Google Cloud **Project ID** here.
        * **`GCP_SA_KEY`**: Open the downloaded JSON key file, copy its entire text content, and paste it here.

---

### 3. Final Permission Step (After Running Code)

The first time you run the setup cells in the notebook, a new BigQuery connection will be created. This connection has its own unique service account that needs permission to use AI models.

1.  **Find the Connection Service Account**
    * After running the setup cells, go to **BigQuery** > **External connections** in your Google Cloud project.
    * Click on the connection named `llm-connection`.
    * Copy its **Service Account ID** (it will look like `bqcx-...@...gserviceaccount.com`).

2.  **Grant Permission**
    * Go to the **IAM & Admin** page.
    * Click **+ Grant Access**.
    * Paste the connection's service account ID into the **"New principals"** box.
    * Give it the single role of **`Vertex AI User`**.
    * Click **Save**.

---

With this setup complete, the notebook has secure access to your Google Cloud project and can run all subsequent analysis cells.

In [2]:
user_secrets = UserSecretsClient()
project_id = user_secrets.get_secret("GCP_PROJECT_ID")
gcp_key_json = user_secrets.get_secret("GCP_SA_KEY")
location = 'US'

In [47]:
# Write the key to a temporary file in the notebook's environment
key_file_path = 'gcp_key.json'
try:
    with open(key_file_path, 'w') as f:
        f.write(gcp_key_json)
    
    # Remove "> /dev/null 2>&1" to show the output.
    # Authenticate the gcloud tool using the key file
    !gcloud auth activate-service-account --key-file={key_file_path} > /dev/null 2>&1
    
    # Configure the gcloud tool to use your project
    !gcloud config set project {project_id} > /dev/null 2>&1
    
finally:
    # Securely delete the key file immediately after use
    if os.path.exists(key_file_path):
        os.remove(key_file_path)

# Enable the Vertex AI and BigQuery Connection APIs. Run only once Or Enable using the Cloud Interface.
# !gcloud services enable aiplatform.googleapis.com bigqueryconnection.googleapis.com > /dev/null 2>&1

In [1]:
# This command creates the connection resource. Remove "> /dev/null 2>&1" to show the output.
!bq mk --connection --location={location} --connection_type=CLOUD_RESOURCE llm-connection > /dev/null 2>&1

In [2]:
# This command shows the details of your connection. Remove "> /dev/null 2>&1" to show the output.
!bq show --connection --location={location} llm-connection > /dev/null 2>&1

# BigQuery Resource Creation

This section runs three commands to create the necessary resources for our analysis inside your BigQuery project.

---

### 1. Create a Dataset in the Correct Region

First, we create a new dataset named `patent_analysis` in our chosen region. This dataset acts as a container for the AI model and the object table we will create next.

### 2. Create a Reference to the AI Model

Next, we create a "shortcut" to Google's `gemini-1.5-flash` model. This command gives us an easy name, `gemini_flash_analyzer`, to use in our analysis queries.

### 3. Create an Object Table for the PDFs

Finally, we create an object table named `patent_documents_object_table`. This is a special "map" that points directly to all the raw PDF files in the public Google Cloud Storage bucket, making them ready for analysis.

---

In [4]:
client = bigquery.Client(project=project_id, location=location)
client

In [17]:
# 1. Create the new dataset in the correct location
create_dataset_query = f"""
CREATE SCHEMA IF NOT EXISTS `{project_id}.patent_analysis`
OPTIONS(location = '{location}');
"""

print(f"Creating dataset 'patent_analysis' in {location}...")
job = client.query(create_dataset_query)
try:
    job.result()
    print("✅ Dataset created successfully or already exists.")
except Exception as e:
    print(f"❌ FAILED to create dataset. Error:\n\n{e}")


# 2. Create the AI model reference inside the new dataset
create_model_query = f"""
CREATE OR REPLACE MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`
  REMOTE WITH CONNECTION `{location}.llm-connection`
  OPTIONS (endpoint = 'gemini-2.5-flash');
"""

print("\nCreating the AI model reference...")
job = client.query(create_model_query)
try:
    job.result() # This waits for the job to complete.
    print("✅ SUCCESS: Model 'gemini_vision_analyzer' created successfully.")
except Exception as e:
    print(f"❌ FAILED: The query failed. Please share this full error message:\n\n{e}")

# TODO: replace the gcs files source with a variable
# 3. Create the Object Table
# This query creates the "map" to the PDF files inside the local 'patent_analysis' dataset.
object_table_query = f"""
CREATE OR REPLACE EXTERNAL TABLE `{project_id}.patent_analysis.patent_documents_object_table`
WITH CONNECTION `{location}.llm-connection`
OPTIONS (
    object_metadata = 'SIMPLE',
    uris = ['gs://gcs-public-data--labeled-patents/*.pdf'] 
);
"""

print("Creating the object table...")
job = client.query(object_table_query)
try:
    job.result()
    print("✅ Object table created successfully.")
except Exception as e:
    print(f"❌ FAILED to create the object table. Error:\n\n{e}")


Creating the AI model reference...
✅ SUCCESS: Model 'gemini_vision_analyzer' created successfully.


# AI-Powered Data Enrichment

With our setup complete, we now begin the core analysis. This phase will extract the data from both the text and diagrams within the patent PDFs.

---

### Step 1: Automated Text & Multilingual Translation

In this first step, we use our `gemini-2.5-flash` model to process all the PDF documents from our object table. The prompt instructs the AI to perform several tasks:

* **Extract** key fields like the title, inventor, abstract, applicant, and relevant dates.
* **Identify** the original language of the document.
* **Translate** the title and abstract into English if the original is in German or French.

The query then parses the AI's structured JSON response and saves the clean, translated results into a new table named `ai_extraction_text`.

In [ ]:
prompt_text = """From this patent document, perform the following tasks:

1.  **Extract these fields**: title, inventor, abstract, 
    the **Filed**, the **Date of Patent**, the international classification code, and the applicant.
    
2.  **Translate**: If the original title and abstract are in German or French, translate them into English.

3.  **Identify Language**: Determine the original language of the document.

Return ONLY a valid JSON object with EXACTLY these ten keys: 
"title_en", "inventor", "abstract_en", "filed", "date_of_patent", "class_international", "applicant", and "original_language".

**Formatting Rule**: For any key that has multiple values (like "inventor" or "class_international" or "applicant"), 
combine them into a single string, separated by a comma and a space. For example: "Igor Karp, Lev Stesin".

The "original_language" value must be one of these three strings: 'EN', 'FR', or 'DE'.
If any other field is unavailable, use null as the value.
"""

# The main SQL query.
sql_query = f"""
CREATE OR REPLACE TABLE `{project_id}.patent_analysis.ai_text_extraction` AS (
  WITH raw_json AS (
      SELECT
        uri,
        ml_generate_text_llm_result AS llm_result
      FROM
        ML.GENERATE_TEXT(
          MODEL `{project_id}.patent_analysis.gemini_vision_analyzer`,
          TABLE `{project_id}.patent_analysis.patent_documents_object_table`,
          STRUCT(
            '''{prompt_text}''' AS prompt,
            2048 AS max_output_tokens,
            0.2 AS temperature,
            TRUE AS flatten_json_output
          )
        )
    ),
    parsed_json AS (
      -- Step 2: Clean and parse the JSON output.
      SELECT
        uri,
        llm_result,
        SAFE.PARSE_JSON(
          REGEXP_REPLACE(llm_result, r'(?s)```json\\n(.*?)\\n```', r'\\1')
        ) AS json_data
      FROM
        raw_json
    )
  SELECT
    uri,
    llm_result,
    
    SAFE.JSON_VALUE(json_data, '$.original_language') AS original_language,
    SAFE.JSON_VALUE(json_data, '$.title_en') AS extracted_title_en,
    SAFE.JSON_VALUE(json_data, '$.inventor') AS extracted_inventor,
    SAFE.JSON_VALUE(json_data, '$.abstract_en') AS extracted_abstract_en,
    SAFE.JSON_VALUE(json_data, '$.filed') AS filed_date,
    SAFE.JSON_VALUE(json_data, '$.date_of_patent') AS official_patent_date,
    SAFE.JSON_VALUE(json_data, '$.class_international') AS class_international,
    SAFE.JSON_VALUE(json_data, '$.applicant') AS applican
    
  FROM
    parsed_json
);
"""

# Run the query to create the table.
print("Running multilingual text extraction on all 403 PDFs... This may take several minutes.")
job = client.query(sql_query)

try:
    job.result()
except Exception as e:
    print(f"❌ FAILED: The query failed. Error:\n\n{e}")